In [15]:
import pandas as pd
import numpy as np

def gen_return_place(player, path, shotType='all'):
    # Load the data
    events = pd.read_csv(path)
    events['pointWonBy'] = events.groupby('pointNumber')['pointWonBy'].bfill()

    # Filter for the player's returns and shots in rally
    returns_place = events[(events['shotHitBy'] == player) & (events['shotInRally'] == 2)][['shotHitBy', 'shotLocationX', 'shotLocationY', 'pointWonBy', 'isWinner', 'shotFhBh', 'lastShotError']].dropna(subset=['pointWonBy']).copy()

    # Adjust shotLocationY and shotLocationX
    returns_place['shotLocationY'] = returns_place['shotLocationY'].apply(lambda y: -y if y > 0 else y)
    returns_place['shotLocationX'] = returns_place.apply(lambda row: -row['shotLocationX'] if row['shotLocationY'] < 0 else row['shotLocationX'], axis=1)
    
    # Filter by shotType if not 'all'
    if shotType != 'all':
        returns_place = returns_place[returns_place['shotFhBh'] == shotType]

    # Categorize into 'left', 'mid', 'right' based on shotLocationX
    returns_place['width'] = returns_place['shotLocationX'].apply(
        lambda x: 'left' if x <= -52.5 else 'mid' if -52.5 < x < 52.5 else 'right'
    )

    # Calculate count + win pct.
    distribution = returns_place.groupby('width').apply(
        lambda df: pd.Series({
            'freq': len(df),
            'win_percentage': int((df['pointWonBy'] == df['shotHitBy']).mean() * 100)
        })
    ).reset_index()

    max_win_percentage = distribution['win_percentage'].max()
    min_win_percentage = distribution['win_percentage'].min()

    # Assign 'max', 'min', or 'no' to the distribution based on win_percentage
    distribution['maxMin'] = distribution['win_percentage'].apply(
        lambda x: 'max' if x == max_win_percentage else 'min' if x == min_win_percentage else 'no'
    )

    # Convert win_percentage to string for display
    distribution['win_percentage'] = distribution['win_percentage'].astype(str) + '%'

    # Adjust x_mapping to match the width values
    x_mapping = {
        'left': {'x': -100},
        'mid': {'x': 0},
        'right': {'x': 100}
    }

    distribution['x'] = distribution['width'].map(lambda d: x_mapping[d]['x'])

    # Export the data as JSON
    returns_place_json = returns_place.to_json(orient='records')
    distribution_json = distribution.to_json(orient='records')

    with open('returns_place.json', 'w') as f:
        f.write(returns_place_json)

    with open('distribution.json', 'w') as f:
        f.write(distribution_json)


/var/folders/gc/y_rdwxcs2pb0n4dz4z33dcth0000gn/T/ipykernel_23973/3281799175.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  distribution = returns_place.groupby('width').apply(


In [ ]:
player = "Elise Wagle"
path = '../Shot_Visuals_EliseWagle_MotkaKomac.csv'
shotType = "all"
gen_return_place(player, path, shotType)